# **EDA**

## IMPORTS

In [133]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
import numpy as np

## Import data

In [134]:
data_path = Path('../data/cleaned')
data_files = list(data_path.glob('*.csv'))
print(f'{len(data_files)} files found:\n')
for file in data_files:
    print(f'\t-{file.name}')

3 files found:

	-cleaned_commits.csv
	-cleaned_info.csv
	-cleaned_issues.csv


In [135]:
for file in data_files:
    if 'commits' in file.name:
        commits_df = pd.read_csv(data_path/file.name, parse_dates=['author_date', 'commit_date'])
    elif 'issues' in file.name:
        issues_df = pd.read_csv(data_path/file.name, parse_dates=['created_at', 'closed_at', 'updated_at'])
    elif 'info' in file.name:
        info_df = pd.read_csv(data_path/file.name, parse_dates=['created_at', 'updated_at', 'pushed_at' ])


In [136]:
commits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   repo_name       500 non-null    object             
 1   sha             500 non-null    object             
 2   author_name     500 non-null    object             
 3   author_email    500 non-null    object             
 4   author_date     500 non-null    datetime64[ns, UTC]
 5   committer_name  500 non-null    object             
 6   commit_date     500 non-null    datetime64[ns, UTC]
 7   message         500 non-null    object             
 8   message_length  500 non-null    int64              
dtypes: datetime64[ns, UTC](2), int64(1), object(6)
memory usage: 35.3+ KB


In [137]:
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   repo_name            500 non-null    object             
 1   issue_id             500 non-null    int64              
 2   issue_number         500 non-null    int64              
 3   title                500 non-null    object             
 4   state                500 non-null    object             
 5   user_login           500 non-null    object             
 6   created_at           500 non-null    datetime64[ns, UTC]
 7   updated_at           500 non-null    datetime64[ns, UTC]
 8   closed_at            250 non-null    datetime64[ns, UTC]
 9   is_open              500 non-null    bool               
 10  comments             500 non-null    int64              
 11  author_association   500 non-null    object             
 12  labels_count         5

In [138]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   repo_name    5 non-null      object             
 1   stars        5 non-null      int64              
 2   forks        5 non-null      int64              
 3   open_issues  5 non-null      int64              
 4   language     5 non-null      object             
 5   description  5 non-null      object             
 6   created_at   5 non-null      datetime64[ns, UTC]
 7   updated_at   5 non-null      datetime64[ns, UTC]
 8   pushed_at    5 non-null      datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), int64(3), object(3)
memory usage: 492.0+ bytes


## Repo summary

In [139]:
# Group issue data by repo_name and perform summary stats:

issues_summary = issues_df.groupby('repo_name').agg({
    'issue_id': 'count',
    'issue_age_days': ['mean', 'median']
})

issues_summary = issues_summary.reset_index() 
issues_summary.columns = (
    f'{top_level}_{bottom_level}' for top_level, bottom_level in issues_summary.columns
)
issues_summary

,repo_name_,issue_id_count,issue_age_days_mean,issue_age_days_median
0,apache/airflow,100,7.18,7.0
1,mlflow/mlflow,100,11.70,6.0
2,numpy/numpy,100,12.42,5.0
3,pandas-dev/pandas,100,12.12,10.5
4,scikit-learn/scikit-learn,100,15.28,6.0


In [140]:
# Group commit data by repo_name and perform aggregation:

commits_summary = commits_df.groupby('repo_name').agg({
    'sha': 'count'
})
commits_summary = commits_summary.reset_index().rename(columns ={'sha': 'commit_sha_count','repo_name': 'repo_name_'})
commits_summary


,repo_name_,commit_sha_count
0,apache/airflow,100
1,mlflow/mlflow,100
2,numpy/numpy,100
3,pandas-dev/pandas,100
4,scikit-learn/scikit-learn,100


In [141]:
# Group info data by repo_name and perform aggregation:

info_summary = info_df.groupby('repo_name').agg({
    'stars': 'sum'
})

info_summary = info_summary.reset_index().rename(columns={'stars': 'starsgazers_sum'})
info_summary

,repo_name,starsgazers_sum
0,apache/airflow,43145
1,mlflow/mlflow,22892
2,numpy/numpy,30782
3,pandas-dev/pandas,47063
4,scikit-learn/scikit-learn,63980


In [142]:
# join all the summaries into one summary:

summary_df = pd.merge(issues_summary, commits_summary, on='repo_name_', how='inner').rename(columns={'repo_name_': 'repo_name'})
summary_df = pd.merge(summary_df, info_summary, on='repo_name', how='inner' )
summary_df

,repo_name,issue_id_count,issue_age_days_mean,issue_age_days_median,commit_sha_count,starsgazers_sum
0,apache/airflow,100,7.18,7.0,100,43145
1,mlflow/mlflow,100,11.70,6.0,100,22892
2,numpy/numpy,100,12.42,5.0,100,30782
3,pandas-dev/pandas,100,12.12,10.5,100,47063
4,scikit-learn/scikit-learn,100,15.28,6.0,100,63980


In [143]:
#The number of unique contributors:
commits_df['author_name'].nunique()

153

In [144]:
#Top 5 contributors:
print('--- Top five Contributors ---')
commits_df['author_name'].value_counts().head(5)

--- Top five Contributors ---


author_name
jorenham            26
Copilot             24
jbrockmendel        22
Daniel Lok          16
Joren Hammudoglu    15
Name: count, dtype: int64

In [145]:
# Check the proportion of closed versus open issues:
issues_df['state'].value_counts(normalize=True)

state
open      0.5
closed    0.5
Name: proportion, dtype: float64

In [146]:
#Check number of closed vs open issues:
issues_df['state'].value_counts()

state
open      250
closed    250
Name: count, dtype: int64

In [147]:
#create new column, closed_issues to hold the time it took to close issues (only actually closed issues):
closed_issues = issues_df[['repo_name', 'time_to_close_hours']].dropna(subset=['time_to_close_hours'])

#Group time to close issues by 'repo_name' and calculate 'average_time_to_close issues' for each rep:
closed_issues = closed_issues.groupby('repo_name').agg({
    'time_to_close_hours': 'mean',
}).reset_index().rename(columns={'time_to_close_hours': 'average_time_to_close_hours'})

closed_issues # Its already sorted in ascending order

,repo_name,average_time_to_close_hours
0,apache/airflow,77.385149
1,mlflow/mlflow,91.657217
2,numpy/numpy,98.301624
3,pandas-dev/pandas,146.507114
4,scikit-learn/scikit-learn,116.270159


In [148]:
#Create resolve_time_cmpr to hold the time it took to close issues and the author association of closed issues:
resolve_time_cmpr = issues_df[['repo_name', 'time_to_close_hours', 'author_association']].dropna(subset=['time_to_close_hours'])

#Group resolve_time_cmpr by repo_name and author_association and then find the mean and median time for each author_association and repo:
resolve_time_cmpr = resolve_time_cmpr.groupby(['repo_name','author_association']).agg({
    'time_to_close_hours':['mean', 'median']
})

# Flatten new columns:
resolve_time_cmpr.columns = (
    f'{top_level}_{bottom_level}' for top_level, bottom_level in resolve_time_cmpr.columns
)

resolve_time_cmpr = resolve_time_cmpr.rename(columns={
    'time_to_close_hours_mean': 'avg_time_to_close',
    'time_to_close_hours_median': 'median_time_to_close'
})

resolve_time_cmpr

avg_time_to_close  \
repo_name                 author_association                      
apache/airflow            COLLABORATOR               168.833611   
                          CONTRIBUTOR                 63.365262   
                          MEMBER                     117.105000   
                          NONE                        92.533854   
mlflow/mlflow             COLLABORATOR               102.563667   
                          CONTRIBUTOR                 78.401883   
                          MEMBER                       1.800926   
                          NONE                       100.455497   
numpy/numpy               CONTRIBUTOR                141.092037   
                          MEMBER                     111.075988   
                          NONE                        62.102768   
pandas-dev/pandas         CONTRIBUTOR                130.148948   
                          MEMBER                     180.093981   
                          NONE                       143.321570   
scikit-learn/scikit-learn CONTRIBUTOR                148.077564   
                          MEMBER                     155.067925   
                          NONE                        41.942431   

                                              median_time_to_close  
repo_name                 author_association                        
apache/airflow            COLLABORATOR                  168.833611  
                          CONTRIBUTOR                    37.834861  
                          MEMBER                        117.105000  
                          NONE                           92.560833  
mlflow/mlflow             COLLABORATOR                   44.933056  
                          CONTRIBUTOR                    39.375000  
                          MEMBER                          2.062778  
                          NONE                           51.676528  
numpy/numpy               CONTRIBUTOR                    45.389444  
                          MEMBER                         25.186667  
                          NONE                           36.644167  
pandas-dev/pandas         CONTRIBUTOR                    98.977500  
                          MEMBER                        162.783333  
                          NONE                           32.611111  
scikit-learn/scikit-learn CONTRIBUTOR                    84.578333  
                          MEMBER                         72.014444  
                          NONE                           19.630000

In [199]:
#create commits_by_date to store repo_name, author_date and message_length:
commits_by_date = commits_df[['repo_name', 'author_date','message_length' ]].copy()

commits_by_date['day_of_the_week'] = commits_by_date['author_date'].dt.day_name()

commits_by_date['author_date'] = commits_by_date['author_date'].dt.date

#Create commits_per_day_df to hold the count of the number of commits per day by repo_name:
commits_per_day = commits_by_date.groupby(['repo_name', 'author_date']).size()

commits_per_day_df = commits_per_day.reset_index(name='number_of_commits')

commits_per_day_df = commits_per_day_df.set_index(['repo_name', 'author_date'])

commits_per_day_df.sort_values(by='number_of_commits', ascending=False)

number_of_commits
repo_name                 author_date                   
apache/airflow            2025-11-07                  66
mlflow/mlflow             2025-11-07                  23
                          2025-11-03                  19
                          2025-11-05                  18
pandas-dev/pandas         2025-11-04                  17
...                                                  ...
                          2025-11-02                   1
scikit-learn/scikit-learn 2025-10-22                   1
                          2025-10-15                   1
                          2025-11-01                   1
                          2025-11-10                   1

[67 rows x 1 columns]